In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessio

/kaggle/input/triples-0-2000/triples_0_2000/question_1807.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_1420.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_213.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_1905.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_581.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_1754.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_180.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_686.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_307.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_350.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_1357.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_1999.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_578.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_536.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_987.txt
/kaggle/input/triples-0-2000/triples_0_2000/question_671.txt
/kaggle/input/trip

In [2]:
! pip install -U sentence-transformers
! pip install langchain 
! pip install chromadb 
! pip install InstructorEmbedding
! pip install tiktoken
! pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently

In [3]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sentence_transformers import SentenceTransformer,util
import json
from langchain.document_loaders import CSVLoader,TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GooglePalm
from langchain.embeddings import GooglePalmEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

from InstructorEmbedding import INSTRUCTOR
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time
from tqdm.notebook import tqdm
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR

# Loading the Dataset

In [4]:
dataset = load_dataset("AmazonScience/mintaka",trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
print(dataset["test"][0])

{'id': 'fae46b21', 'lang': 'en', 'question': 'What man was a famous American author and also a steamboat pilot on the Mississippi River?', 'answerText': 'Mark Twain', 'category': 'history', 'complexityType': 'intersection', 'questionEntity': [{'name': 'Q1497', 'entityType': 'entity', 'label': 'Mississippi River', 'mention': 'Mississippi River', 'span': [72, 89]}, {'name': 'Q846570', 'entityType': 'entity', 'label': 'Americans', 'mention': 'American', 'span': [22, 30]}], 'answerEntity': [{'name': 'Q7245', 'label': 'Mark Twain'}]}


In [6]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

data = dataset['test']

dataset2 = MyDataset(data)

df = pd.DataFrame([sample for sample in dataset2])
df

,id,lang,question,answerText,category,complexityType,questionEntity,answerEntity
0,fae46b21,en,What man was a famous American author and also...,Mark Twain,history,intersection,"[{'name': 'Q1497', 'entityType': 'entity', 'la...","[{'name': 'Q7245', 'label': 'Mark Twain'}]"
1,bc8713cc,en,How many Academy Awards has Jake Gyllenhaal be...,1,movies,count,"[{'name': 'Q133313', 'entityType': 'entity', '...","[{'name': 'Q106291', 'label': 'Academy Award f..."
2,d2a03f72,en,"Who is older, The Weeknd or Drake?",Drake,music,comparative,"[{'name': 'Q2121062', 'entityType': 'entity', ...","[{'name': 'Q33240', 'label': 'Drake'}]"
3,9a296167,en,How many children did Donald Trump have?,5,history,count,"[{'name': 'Q22686', 'entityType': 'entity', 'l...","[{'name': 'Q3713655', 'label': 'Donald Trump J..."
4,e343ad26,en,Is the main hero in Final Fantasy IX named Kuja?,No,videogames,yesno,"[{'name': 'Q474573', 'entityType': 'entity', '...",[]
...,...,...,...,...,...,...,...,...
3995,d52b03ee,en,Who was the first woman mayor of San Francisco?,Dianne Feinstein,politics,ordinal,"[{'name': 'Q62', 'entityType': 'entity', 'labe...","[{'name': 'Q230733', 'label': 'Dianne Feinstei..."
3996,07f2947c,en,Where was the last Republican mayor of Boston ...,"Portland, Maine",politics,ordinal,"[{'name': 'Q100', 'entityType': 'entity', 'lab...","[{'name': 'Q49201', 'label': 'Portland'}]"
3997,58727fb0,en,How long was the 53rd mayor of Boston in office?,"20 years, 6 months",politics,ordinal,"[{'name': 'Q100', 'entityType': 'entity', 'lab...",[]
3998,a25818a2,en,Who was the first British monarch to have a pr...,George I,politics,ordinal,"[{'name': 'Q145', 'entityType': 'entity', 'lab...","[{'name': 'Q130805', 'label': 'George I of Gre..."


In [7]:
embedding = SentenceTransformerEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs={"device":"cuda"})

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [8]:
OUT_PATH="/kaggle/input/triples-2000-4000/extracted"

In [9]:
def create_embeddings(question_id):
    try:
        loader = TextLoader(f"{OUT_PATH}/question_{question_id}.txt")
        documents = loader.load()
    except Exception as e:
        print(e)
        return
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
    texts = text_splitter.split_documents(documents)
    db_embeddings = FAISS.from_documents(texts,embedding)
    db_embeddings.save_local(f"/kaggle/working/embeddings/question_{question_id}")
    print("After Saving")
    

 

In [10]:
start  = 1943
end = 2000
for i in range(start,end):
    create_embeddings(i)
    print(f"Question {i} Complete")
    
    

After Saving
Question 1943 Complete
After Saving
Question 1944 Complete
After Saving
Question 1945 Complete
After Saving
Question 1946 Complete
After Saving
Question 1947 Complete
After Saving
Question 1948 Complete
After Saving
Question 1949 Complete
After Saving
Question 1950 Complete
After Saving
Question 1951 Complete
After Saving
Question 1952 Complete
After Saving
Question 1953 Complete
After Saving
Question 1954 Complete
After Saving
Question 1955 Complete
After Saving
Question 1956 Complete
After Saving
Question 1957 Complete
After Saving
Question 1958 Complete
After Saving
Question 1959 Complete
After Saving
Question 1960 Complete
After Saving
Question 1961 Complete
After Saving
Question 1962 Complete
After Saving
Question 1963 Complete
After Saving
Question 1964 Complete
After Saving
Question 1965 Complete
After Saving
Question 1966 Complete
After Saving
Question 1967 Complete
After Saving
Question 1968 Complete
After Saving
Question 1969 Complete
After Saving
Question 1970 C

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()